In [1]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

## 3 CLASS

In [2]:
mias = pd.read_csv("E:/WORKBASE/Project-rsna-breast-cancer-detection/DATA_STANDARD/CSV 3 CLASS/Mias.csv", index_col='Unnamed: 0')

In [3]:
ddsm = pd.read_csv("E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CSV 3 CLASS\meta_ddsm.csv", index_col='Unnamed: 0')

In [4]:
cdd = pd.read_csv("E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CSV 3 CLASS\meta_cdd.csv", index_col='Unnamed: 0')

In [5]:
cmmd = pd.read_csv("E:/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/meta_main.csv", index_col='Unnamed: 0')
cmmd.path[0]

'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/TheChineseMammographyDatabase/TheChineseMammographyDatabase/CMMD/D1-0001/07-18-2010-NA-NA-79377/1.000000-NA-70244\\1-1.dcm'

In [6]:
cmmd.path = cmmd.path.apply(lambda x: x.replace('D:/OneDrive - Industrial University of HoChiMinh City', 'E:'))

In [7]:
cmmd.image_id = cmmd.image_id.apply(lambda x: x.replace('_', '-'))

In [8]:
cmmd

,index,patient_id,image_id,laterality,cancer,path,view
0,0,D1-0001,D1-0001-1-1,R,0,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,MLO
1,1,D1-0001,D1-0001-1-2,R,0,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,CC
2,2,D1-0002,D1-0002-1-1,L,0,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,MLO
3,3,D1-0002,D1-0002-1-2,L,0,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,CC
4,4,D1-0003,D1-0003-1-1,L,0,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,MLO
...,...,...,...,...,...,...,...
3736,3736,D2-0747,D2-0747-1-4,R,1,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,CC
3737,3737,D2-0748,D2-0748-1-3,R,1,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,MLO
3738,3738,D2-0748,D2-0748-1-4,R,1,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,CC
3739,3739,D2-0749,D2-0749-1-1,L,1,E:/WORKBASE/Project-rsna-breast-cancer-detecti...,MLO


In [9]:
rsna = pd.read_csv("E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\RSNA.csv", index_col='Unnamed: 0')

> MOVE IMAGE MISTASK

In [10]:
from remove_image_views import *


In [25]:
mv_cmmd = move_image(cmmd, path='E:/WORKBASE/Project-rsna-breast-cancer-detection/DATA_STANDARD/Croped_CMMD')
mv_cmmd.my_method(cmmd.index)
# mv_cmmd.process_move(0)

In [ ]:
mv_rsna = move_image(rsna, path='E:/WORKBASE/Project-rsna-breast-cancer-detection/DATA_STANDARD/Croped_RSNA')
mv_rsna.my_method(rsna.index)
# mv_cmmd.process_move(0)

In [104]:
def procees_dcm(path):
    
    dicom = pydicom.dcmread(path)
    # try : 
    img = dicom.pixel_array
    # img = apply_windowing(img, dicom)
    img = (img - img.min()) / (img.max() - img.min())

    if dicom.PhotometricInterpretation == "MONOCHROME1":  
        img = 1 - img
    image = np.uint8(img * 255)

    return image

In [108]:
for id in cmmd.index:
    par = cmmd.patient_id[id]
    lat = cmmd.laterality[id]
    view = cmmd.view[id]
    file_name = f"E:/WORKBASE/Project-rsna-breast-cancer-detection/DATA_STANDARD/Croped_CMMD/{cmmd.view[id]}_{cmmd.image_id[id]}.png"
    if os.path.exists(file_name) ==False:
        imageio.imwrite(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/DATA_STANDARD/Croped_CMMD/{view}_{cmmd.image_id[id]}.png', procees_dcm(cmmd.path[id]))

> ADD PATH

In [5]:
mias['Path'] = mias.image_id.apply(lambda x: f'Data_image/Mias/{x}.png')

In [6]:
ddsm['Path'] = ddsm.apply(lambda row: f'Data_image/{row["patient_id"]}/{row["view"]}/{row["laterality"]}/{row["view"]}_{row["laterality"]}_{row["image_id"]}.png', axis=1)

In [7]:
cdd['Path'] = cdd.apply(lambda row: f'Data_image/{row["patient_id"]}/{row["view"]}/{row["laterality"]}/{row["view"]}_{row["laterality"]}_{row["image_id"]}.png', axis=1)
cdd

,index,patient_id,image_id,view,laterality,status,density,birads,age,Path
0,0,1CDD,P1-L-DM-MLO,MLO,L,2,C,4,46,Data_image/1CDD/MLO/L/MLO_L_P1-L-DM-MLO.png
1,2,2CDD,P2-R-DM-CC,CC,R,1,C,2,31,Data_image/2CDD/CC/R/CC_R_P2-R-DM-CC.png
2,3,2CDD,P2-R-DM-MLO,MLO,R,1,C,2,31,Data_image/2CDD/MLO/R/MLO_R_P2-R-DM-MLO.png
3,6,2CDD,P2-L-DM-CC,CC,L,1,C,2,31,Data_image/2CDD/CC/L/CC_L_P2-L-DM-CC.png
4,7,2CDD,P2-L-DM-MLO,MLO,L,1,C,2,31,Data_image/2CDD/MLO/L/MLO_L_P2-L-DM-MLO.png
...,...,...,...,...,...,...,...,...,...,...
1061,2076,316CDD,P316-L-DM-MLO,MLO,L,2,B,2,68,Data_image/316CDD/MLO/L/MLO_L_P316-L-DM-MLO.png
1062,2079,319CDD,P319-R-DM-CC,CC,R,1,B,3,46,Data_image/319CDD/CC/R/CC_R_P319-R-DM-CC.png
1063,2080,319CDD,P319-R-DM-MLO,MLO,R,1,B,3,46,Data_image/319CDD/MLO/R/MLO_R_P319-R-DM-MLO.png
1064,2081,325CDD,P325-R-DM-CC,CC,R,1,B,3,49,Data_image/325CDD/CC/R/CC_R_P325-R-DM-CC.png


> ADD TYPE DATA

In [8]:
mias['Type'] = mias.image_id.apply(lambda x: 'Mias')
ddsm['Type'] = ddsm.image_id.apply(lambda x: 'ddsm')
cdd['Type'] = cdd.image_id.apply(lambda x: 'cdd')

> Format data

In [9]:
mias = mias[['image_id', 'Path', 'Type', 'status']]
cdd = cdd[['image_id', 'Path', 'Type', 'status']]
ddsm = ddsm[['image_id', 'Path', 'Type', 'status']]

> Split data

In [10]:
print(f'Size Mias {len(mias)}')
print(f'Size cdd {len(cdd)}')
print(f'Size ddsm {len(ddsm)}')

Size Mias 322
Size cdd 1066
Size ddsm 7808


In [11]:
print(f'Detail mias {mias.status.value_counts()}')
print(f'Detail cdd {cdd.status.value_counts()}')
print(f'Detail ddsm {ddsm.status.value_counts()}')

Detail mias status
0    207
1     63
2     52
Name: count, dtype: int64
Detail cdd status
2    364
1    362
0    340
Name: count, dtype: int64
Detail ddsm status
2    2716
1    2684
0    2408
Name: count, dtype: int64


TEST

In [39]:
def get10(data, ts1, ts2, ts3):
    d0 = data[data.status==0].sample(ts1)
    d1 = data[data.status==1].sample(ts2)
    d2 = data[data.status==2].sample(ts3)
    return pd.concat([d0, d1, d2]).sample(frac=1).reset_index(drop=True)
def concatdata(list_data):
    # meta = pd.DataFrame()
    list_idimage = []
    for data, name,  ts1, ts2, ts3 in list_data:
        # if len(meta) == 0:
        meta = get10(data, ts1, ts2, ts3)
        list_idimage = list_idimage + list(meta.image_id)
        meta.to_csv(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/3CLASS/Test_{name}.csv')
    return list_idimage 

In [41]:
list_idimage = concatdata([(mias, 'mias', 20, 10, 10), (ddsm, 'ddsm', 271, 271, 271), (cdd, 'cdd', 36, 36, 36)])

In [43]:
meta = pd.concat([mias, ddsm, cdd]).sample(frac=1).reset_index(drop=True)
meta = meta[~meta.image_id.isin(list_idimage)]
meta

,image_id,Path,Type,status
0,4580DDSM,Data_image/3053DDSM/CC/L/CC_L_4580DDSM.png,ddsm,2
2,2338DDSM,Data_image/4010DDSM/CC/R/CC_R_2338DDSM.png,ddsm,1
3,2764DDSM,Data_image/0023DDSM/CC/L/CC_L_2764DDSM.png,ddsm,2
4,139DDSM,Data_image/0276DDSM/MLO/R/MLO_R_139DDSM.png,ddsm,1
5,7128DDSM,Data_image/1963DDSM/CC/L/CC_L_7128DDSM.png,ddsm,0
...,...,...,...,...
9191,4864DDSM,Data_image/3403DDSM/CC/L/CC_L_4864DDSM.png,ddsm,2
9192,5937DDSM,Data_image/0225DDSM/MLO/L/MLO_L_5937DDSM.png,ddsm,0
9193,7386DDSM,Data_image/2039DDSM/CC/R/CC_R_7386DDSM.png,ddsm,0
9194,mdb255,Data_image/Mias/mdb255.png,Mias,0


In [44]:
from sklearn.model_selection import KFold


In [59]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Lặp lại huấn luyện và đánh giá trên các phần được chia
for id, (train_index, val_index) in enumerate(kf.split(meta)):
    # Tạo dữ liệu huấn luyện và dữ liệu đánh giá
    os.makedirs(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/3CLASS/{id}-fold')
    
    train_data = meta.iloc[train_index]
    val_data = meta.iloc[val_index]
    train_data.to_csv(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/3CLASS/{id}-fold/train.csv')
    val_data.to_csv(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/3CLASS/{id}-fold/val.csv')
    ## description
    with open(f'E:/WORKBASE/Project-rsna-breast-cancer-detection/META_DATA/3CLASS/{id}-fold/description.txt', 'a') as f:
        f.writelines(f'Size of train: {len(train_data)} \n\n Size of val: {len(val_data)} \n\n Number of per dataset: \n {train_data.Type.value_counts()}\n\n Status of set train: \n{train_data.status.value_counts()}')
    print(len(train_data), len(val_data))
    print(train_data.Type.value_counts())
    print(train_data.status.value_counts())
    print('++++++++++++++++++')

6580 1646
Type
ddsm    5567
cdd      780
Mias     233
Name: count, dtype: int64
status
2    2272
1    2253
0    2055
Name: count, dtype: int64
++++++++++++++++++
6581 1645
Type
ddsm    5599
cdd      752
Mias     230
Name: count, dtype: int64
status
2    2246
1    2231
0    2104
Name: count, dtype: int64
++++++++++++++++++
6581 1645
Type
ddsm    5587
cdd      766
Mias     228
Name: count, dtype: int64
status
2    2259
1    2201
0    2121
Name: count, dtype: int64
++++++++++++++++++
6581 1645
Type
ddsm    5607
cdd      747
Mias     227
Name: count, dtype: int64
status
2    2240
1    2210
0    2131
Name: count, dtype: int64
++++++++++++++++++
6581 1645
Type
ddsm    5620
cdd      751
Mias     210
Name: count, dtype: int64
status
1    2253
2    2227
0    2101
Name: count, dtype: int64
++++++++++++++++++


## 2 CLASS

We have 5 dataset: DDSM, RSNA, CMMD, CDD, Mias
+ So we will collect status 0, 1 --> 0 and 2 --> 1

In [61]:
mias

,image_id,Path,Type,status
0,mdb001,Data_image/Mias/mdb001.png,Mias,1
1,mdb002,Data_image/Mias/mdb002.png,Mias,1
2,mdb003,Data_image/Mias/mdb003.png,Mias,0
3,mdb004,Data_image/Mias/mdb004.png,Mias,0
4,mdb005,Data_image/Mias/mdb005.png,Mias,1
...,...,...,...,...
317,mdb318,Data_image/Mias/mdb318.png,Mias,0
318,mdb319,Data_image/Mias/mdb319.png,Mias,0
319,mdb320,Data_image/Mias/mdb320.png,Mias,0
320,mdb321,Data_image/Mias/mdb321.png,Mias,0


In [62]:
ddsm

,image_id,Path,Type,status
0,0DDSM,Data_image/0029DDSM/CC/L/CC_L_0DDSM.png,ddsm,1
1,1DDSM,Data_image/0029DDSM/MLO/L/MLO_L_1DDSM.png,ddsm,1
2,2DDSM,Data_image/0029DDSM/CC/R/CC_R_2DDSM.png,ddsm,1
3,3DDSM,Data_image/0029DDSM/MLO/R/MLO_R_3DDSM.png,ddsm,1
4,4DDSM,Data_image/0033DDSM/CC/L/CC_L_4DDSM.png,ddsm,1
...,...,...,...,...
7803,7803DDSM,Data_image/4607DDSM/MLO/R/MLO_R_7803DDSM.png,ddsm,0
7804,7804DDSM,Data_image/4608DDSM/CC/L/CC_L_7804DDSM.png,ddsm,0
7805,7805DDSM,Data_image/4608DDSM/MLO/L/MLO_L_7805DDSM.png,ddsm,0
7806,7806DDSM,Data_image/4608DDSM/CC/R/CC_R_7806DDSM.png,ddsm,0


In [63]:
cdd

,image_id,Path,Type,status
0,P1-L-DM-MLO,Data_image/1CDD/MLO/L/MLO_L_P1-L-DM-MLO.png,cdd,2
1,P2-R-DM-CC,Data_image/2CDD/CC/R/CC_R_P2-R-DM-CC.png,cdd,1
2,P2-R-DM-MLO,Data_image/2CDD/MLO/R/MLO_R_P2-R-DM-MLO.png,cdd,1
3,P2-L-DM-CC,Data_image/2CDD/CC/L/CC_L_P2-L-DM-CC.png,cdd,1
4,P2-L-DM-MLO,Data_image/2CDD/MLO/L/MLO_L_P2-L-DM-MLO.png,cdd,1
...,...,...,...,...
1061,P316-L-DM-MLO,Data_image/316CDD/MLO/L/MLO_L_P316-L-DM-MLO.png,cdd,2
1062,P319-R-DM-CC,Data_image/319CDD/CC/R/CC_R_P319-R-DM-CC.png,cdd,1
1063,P319-R-DM-MLO,Data_image/319CDD/MLO/R/MLO_R_P319-R-DM-MLO.png,cdd,1
1064,P325-R-DM-CC,Data_image/325CDD/CC/R/CC_R_P325-R-DM-CC.png,cdd,1
